In [1]:
import requests
import json
import pandas as pd
from dateutil import parser
import matplotlib
import matplotlib.pyplot as plt


KEY = "secret_dhDJCxIGksOzhxiKis5UfH8VWA6JXTnF5T44BANarrG"
headers = {"Authorization":KEY, "Notion-Version":"2022-06-28", "Content-Type":"application/json"}
base_url = "https://api.notion.com/v1/databases/"
publication_database_id = '54cb3960b40246e2be42cfb58e2b45fd'
people_database_id = '778b56a297cc4fd7b45d8f40c3d4e1cb'
query_url = base_url+publication_database_id+"/query"
print("query_url:", query_url)
#filter_rule = {"or": [dict(property="年月", "")]}
filter_rule={}
sort_rules = [dict(property="Date", direction="descending")]

query_data = dict(page_size=100, sorts=sort_rules)#, filter=filter_rule)

#res = requests.post(query_url, headers=headers, query_data=query_data)
#result = res.content
#obj = json.loads( result.decode() )
#df = pd.DataFrame(obj["results"])


query_url: https://api.notion.com/v1/databases/54cb3960b40246e2be42cfb58e2b45fd/query


In [2]:
import copy
def fetch_notion_db(query_url, headers, query_data):
    query_data = copy.deepcopy(query_data)
    result_list = []
    counter = 0
    #query_data["start_cursor"] = "undefined"
    while True:
        print(counter)
        res = json.loads( requests.post(query_url, headers=headers, data=json.dumps(query_data)).content.decode() )
        #print(res)
        has_more = res.get("has_more")
        result_list = result_list + res["results"]
        print(len(result_list))
        print(has_more, res["next_cursor"], query_data.get("start_cursor", None))
        if has_more is None or has_more==False:
            break
        query_data["start_cursor"] = res["next_cursor"]
        #query_data["start_cursor"] = res["next_cursor"]
        counter+=1
    return result_list

datalist = fetch_notion_db(query_url, headers, query_data)
df = pd.DataFrame(datalist)

0
41
False None None


In [3]:
len(df)

41

In [5]:

import os
import urllib.request
from tqdm import tqdm
def gather_plain_text(richtext_obj):
    richtext_list = richtext_obj["rich_text"]
    plain = ""
    for rt in richtext_list:
        plain = plain + rt["plain_text"]
    return plain
def parse_thumbnail(thumb_obj):
    thumbfs  = thumb_obj["files"]
    assert len(thumbfs)>0
    thumbf = thumbfs[0]
    if thumbf["type"]=="external":
        thumbnail_link = thumbf["external"]["url"]
    else:
        thumbnail_link = thumbf["file"]["url"]
    return thumbnail_link
def convert2gruvi(datalist, download_thumbnails=True):
    if download_thumbnails and not os.path.exists("downloads"):
        os.makedirs("downloads")
    all_str = ""
    for i, data in enumerate(datalist):
        dID = data["id"]
        diprop = data["properties"]
        year, title, authors, paper_link, venue = None, None, None, None, None
        if diprop.get("Date", None) is not None:
            date = diprop["Date"]["date"]["start"]
            year = date[:4]
        title = diprop["Name"]["title"][0]["text"]["content"]
        if diprop.get("Authors", None) is not None:
            authors  = gather_plain_text( diprop["Authors"] )
        if diprop.get("paper_link", None) is not None:
            paper_link  = gather_plain_text( diprop["paper_link"] )
        if diprop.get("project_link", None) is not None:
            project_link  = gather_plain_text( diprop["project_link"] )
        if diprop.get("video_link", None) is not None:
            video_link  = gather_plain_text( diprop["video_link"] )
        if diprop.get("presentation_link", None) is not None:
            presentation_link  = gather_plain_text( diprop["presentation_link"] )
        if diprop.get("Venue", None) is not None:
            venue  = gather_plain_text( diprop["Venue"] )
        print(date, title)


        thumbnail_link = parse_thumbnail(diprop["Thumbnail"])
        fsuffix = thumbnail_link.split("?")[0].split(".")[-1]
        thumb_name = date + '_' + dID + '.' + fsuffix 
        if download_thumbnails:
            print(f'Downloading thumbnail {thumb_name} from {thumbnail_link}')
            urllib.request.urlretrieve(thumbnail_link, f"../images/pubpic/{thumb_name}")

        export_str = f"""
- title: >-
    {title}
  image: {thumb_name}
  description: >-
  authors: {authors}
  venue: >-
    {venue}
  year: {year}
  highlight: 1
  pdf: {paper_link}
"""
        if project_link is not None:
            export_str = export_str + f"  project_page: {project_link}\n"
        if video_link is not None and len(video_link)>0:
            export_str = export_str + f"  video: {video_link}\n"
        if presentation_link is not None and len(presentation_link)>0:
            export_str = export_str + f"  presentation: {presentation_link}\n"
        all_str = all_str + export_str+ "\n\n"
    return all_str

all_str = convert2gruvi(datalist, download_thumbnails=True)
with open("../_data/old_manual_publist_stopped_at_2021.yml", "r") as f:
    old_str = f.read()
with open("../_data/publist.yml", "w") as f:
    f.write(all_str + "\n ################################# BELOW: MANUAL ENTRIES ################################# \n" + old_str)
print(all_str)

2023-10-02 UniT3D: A Unified Transformer for 3D Dense Captioning and Visual Grounding


2023-10-02 HAL3D: Hierarchical Active Learning for Fine-Grained 3D Part Labeling
2023-10-02 PARIS: Part-level Reconstruction and Motion Analysis for Articulated Objects
2023-10-02 SKED: Sketch-guided Text-based 3D Editing
2023-10-02 DS-Fusion: Artistic Typography via Discriminated and Stylized Diffusion
2023-10-02 Multi3DRefer: Grounding Text Description to Multiple 3D Objects
2023-07-01 Learning Physically Simulated Tennis Skills from Broadcast Videos
2023-07-01 Synthesizing Physical Character-Scene Interactions
2023-07-01 CALM: Conditional Adversarial Latent Models for Directable Virtual Characters
2023-06-14 Realistic Saliency Guided Image Enhancement
2023-06-14 Computational Flash Photography through Intrinsics
2023-05-01 Emergence of Maps in the Memories of Blind Navigation Agents
2023-04-24 Hierarchical Neural Coding for Controllable CAD Model Generation
2023-04-01 Habitat-Matterport 3D Semantics Dataset
2023-04-01 ARO-Net: Learning Implicit Fields from Anchored Radial Observatio

In [4]:
import os
os.getcwd()

'/local-scratch/localhome/xya120/studio/gruvi/gruvilab.github.io/scripts'

In [3]:
down.keys(), down["next_cursor"]

NameError: name 'down' is not defined

In [1]:
len(down["results"]), down["results"][1]["properties"]

NameError: name 'down' is not defined